In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import texthero as hero
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk

In [ ]:
SFW_16  = pd.read_csv('Enhanced construct - 16 sectors.xlsx - Job Role_TSC.csv', error_bad_lines=False)

In [ ]:
SFW_13  = pd.read_csv('Enhanced construct - 13 sectors.xlsx - Job Role_TSC.csv', error_bad_lines=False)

In [ ]:
SFW_16

In [ ]:
SFW_13

In [ ]:
sfw = pd.concat([SFW_16, SFW_13], sort = False, ignore_index = True)

In [ ]:
sfw

In [ ]:
mydict = np.load('replace_spelling.npy', allow_pickle=True) 

In [ ]:
mydict

In [ ]:
spelling_keys = mydict.item().keys()

In [ ]:
spelling_values = mydict.item().values()

In [ ]:
spelling_dict = dict(zip(spelling_keys, spelling_values))

In [ ]:
def replace_all(text, mydict):
    for gb, us in mydict.items():
        text = text.replace(us, gb)
    return text

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """

    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [ ]:
def new_col(sfw, spelling_dict):
    
    sfw['prof + knowlg + ability'] = ""
    for i in range(0, len(sfw)):
        sfw.loc[i, 'prof + knowlg + ability'] = str(sfw.loc[i, 'Proficiency']) + str(sfw.loc[i, 'Knowledge']) + str(sfw.loc[i, 'Abilities'])
    
    sfw['clean_docs'] = sfw['prof + knowlg + ability'].apply(lambda x: x.replace('\n', '. '))
    sfw['clean_docs'] = sfw['clean_docs'].apply(lambda x: x.replace('•', ' '))
    sfw['clean_docs'] = sfw['prof + knowlg + ability'].apply(lambda x: x.replace('\n', '. '))
    sfw['clean_docs'] = sfw['clean_docs'].apply(lambda x: x.replace('•', ' '))   
    sfw['clean_docs'] = sfw['clean_docs'].apply(lambda x: replace_all(x, spelling_dict))
    sfw['clean_docs'] = sfw['clean_docs'].astype(str).apply(clean_text)
    sfw['Proficiency Level'] = sfw['Proficiency Level'].str.split(n=1).str[1]
    
    return sfw

In [ ]:
sfw = new_col(sfw, spelling_dict)

In [ ]:
sfw = sfw[["Job Role","clean_docs", "Sector", "Skill Title", "Proficiency Level"]]

In [ ]:
sfw

In [ ]:
def concat_values(df):
    jd_list = []
    counter = 0
    for row in df[1]:
        jd=""
        counter+=1
        for val in row:
            jd+=val[0]
        jd_list.append(jd)
    return jd_list

In [ ]:
def create_skill_description(sfw):
    sfw_concatenated = sfw.groupby(['Sector','Skill Title','Proficiency Level'])['clean_docs'].unique()
    sfw_concatenated = pd.DataFrame(sfw_concatenated)
    sfw_concatenated  = sfw_concatenated.groupby(['Sector','Skill Title'])['clean_docs']
    sfw_concatenated = pd.DataFrame(sfw_concatenated)
    
    jd_list = concat_values(sfw_concatenated)
    
    sfw_concatenated['Skill description'] = jd_list
    sfw_concatenated[['Sector', 'Skill Title']] = pd.DataFrame(sfw_concatenated[0].tolist()) 
    sfw_concatenated = sfw_concatenated.drop(columns = [0, 1])
    sfw_concatenated = sfw_concatenated[['Sector', 'Skill Title', 'Skill description']]
    
    sfw_new = sfw.merge(sfw_concatenated, on=["Sector","Skill Title"], how = 'left')

    sf_model_final = sfw_new[['Job Role', 'Sector', 'Skill Title', 'Skill description']]
    
    sf_model_final['Skill description'] = hero.remove_stopwords(sf_model_final['Skill description'].astype(str))
    sf_model_final['Skill description'] = hero.remove_urls(sf_model_final['Skill description'])
    sf_model_final['Skill description'] = hero.tokenize(sf_model_final['Skill description']).astype(str)
    sf_model_final['Skill description'] = hero.clean(sf_model_final['Skill description'])

    return sf_model_final

In [ ]:
sf_model_final = create_skill_description(sfw)

In [ ]:
sf_model_final

In [ ]:
sf_model_final[sf_model_final['Job Role'].str.contains('Head')]

In [ ]:
# merge skills of each job role into a list of skills

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def clear_hierarchy(title):
    
    patterns = ["Senior", "Junior", "Manager", "Executive", "Director", "Lead", "Head of", "(Specialist)", "Specialist"]
    
    for s in patterns:
        title = re.sub(s, '', title)
        
    split = title.split()
    if (len(split)!=0):
        
        if (split[0] == 'Assistant') or (split[0]=='Head'):
            title = " ".join(split[1:])

        if title[-2:]=="or":
            title = title[:-2].strip()+"ing"

    return title

In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [ ]:
def lemmatize_text(text):
    list_ = []
    for w in w_tokenizer.tokenize(text):
        list_.append(lemmatize_stemming(w))
    
    title = ' '.join(list_)
  
    return title

In [ ]:
sf_model_final['Job_Role_Normal'] = sf_model_final['Job Role'].apply(lambda x: clear_hierarchy(x))
sf_model_final['Job_Role_Stemmed'] = sf_model_final.Job_Role_Normal.apply(lemmatize_text)

In [ ]:
sf_model_final = sf_model_final[sf_model_final['Job_Role_Normal']!='']

In [ ]:
sf_model_final

In [ ]:
sf_model_final = sf_model_final.groupby(['Job_Role_Stemmed', 'Sector']).agg({'Job Role': lambda series: list(series), 'Skill description': lambda series: list(series), 'Skill Title':lambda series: list(series)})

In [ ]:
sf_model_final

In [ ]:
sf_model_final.reset_index(inplace=True)

In [ ]:
sf_model_final['Job Role'] = sf_model_final['Job Role'].apply(lambda x: list(dict.fromkeys(x)))

In [ ]:
sf_model_final

In [ ]:
def to_lower(x):
    y = []
    for i in range(len(x)):
        y.append(x[i].lower())
    return y

In [ ]:
sf_model_final['Skill Title'] = sf_model_final['Skill Title'].apply(lambda x: to_lower(x))

In [ ]:
sf_model_final

In [ ]:
sf_model_final['Skill Title'] = sf_model_final['Skill Title'].astype(str)
sf_model_final['Skill Title'] = sf_model_final['Skill Title'].apply(lambda x: x.replace("'", ""))
sf_model_final['Skills_unlisted'] = sf_model_final['Skill Title'].apply(lambda x: x.replace('[','').replace(']','').replace(",", " "))

In [ ]:
sf_model_final

In [ ]:
sf_model_final.to_csv('sf_model_final.csv')